In [87]:
from random import random
from math import exp,pow
import matplotlib.pyplot as plt
import numpy as np
from numba import njit, prange

#@njit(fastmath=True)
def nY_fast(t, T, yT,dyT, DT,l):
    sol = yT # initial conditon
    sol += t**2-T**2 -(t**5-T**5)/5 # source  
    sol += (t-T)*yT**2 # 0 order control
    sol += 2*((t**2-T**2)/2 - T*(t-T))*yT*dyT # 1 order control
    #l = 10 # >1 russian roulette parameter 
    if random()*l < (t-T)/DT:
        S = T + random()*(t-T)  # \sim Uniform(T,t)
        sol += l*DT*(nY_fast(S, T, yT, dyT, DT,l)*nY_fast(S, T, yT, dyT, DT,l) -yT**2-2*(S-T)*yT*dyT) 
        global nrec
        nrec+=1
    return sol 


#@njit(fastmath=True)
def nY_slow(T, DT: "step size slow recursion",l):
    y,t = 0,0
    while t<T:
        tt = t+DT if t+DT<T else T
        dy = y**2 -t**4+2*t 
        y = nY_fast(tt, t, y,dy, tt-t,l) 
        t=tt
        global points
        points += [(t,y)]
    return y


@njit(fastmath=True, parallel=True)
def nsol(T, DT, nsim ):
    total = 0.0
    for _ in prange(nsim):
        total += nY_slow(T,DT)
    return total/nsim

In [89]:
nsim = 64
points =[] 
nrec =0

for _ in range(nsim):
    nY_slow(1,0.2,10)
x,y = zip(*points)
x = np.array(x)+0.03
plt.scatter(x,y,color="blue")
print(f"average nrec={nrec/nsim}")

points =[] 
nrec =0
for _ in range(64):
    nY_slow(1,0.2/3,10)
x,y = zip(*points)
plt.scatter(x,y,color="green")
print(f"average nrec={nrec/(3*nsim)}")

points =[] 
nrec =0

for _ in range(nsim):
    nY_slow(1,0.2,0)
x,y = zip(*points)
x = np.array(x)+0.03
plt.scatter(x,y,color="yellow")
print(f"average nrec={nrec/nsim}")


ll = np.arange(0,1,0.01)
plt.plot(ll,ll**2,color ="red")

plt.show()


TypeError: nY_fast() missing 1 required positional argument: 'l'